# Interactive PreliZ examples


The examples below can be executed from within this page thanks to embedded integration with Binder via Thebe.
Click on the button below and start playing with PreliZ right from your browser!

:::{thebe-button}
:::

In [1]:
import preliz as pz

(roulette_interactive)=
# `preliz.roulette`

In [2]:
pz.roulette()